<a href="https://colab.research.google.com/github/lhtvineettiwari/Large-Language-Models-LLMs/blob/main/Copy_of_koala7b_8bit_huggingfaceembedding_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Create a file name `requirement.txt` and insert the code
```
langchain==0.0.142
llama_index==0.5.17
```

In [ ]:
!pip install -r requirements.txt

In [ ]:
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip -q install datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install transformers sentence_transformers 

In [ ]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
import torch
from llama_index import SimpleDirectoryReader, GPTSimpleVectorIndex, PromptHelper, LLMPredictor, ServiceContext, LangchainEmbedding
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.llms.base import LLM

In [ ]:
tokenizer = LlamaTokenizer.from_pretrained("samwit/koala-7b")
model = LlamaForCausalLM.from_pretrained("samwit/koala-7b",load_in_8bit=True,device_map='auto',)
pipeline = pipeline("text-generation",model=model, tokenizer=tokenizer, max_length=512,temperature=0.7,top_p=0.95,repetition_penalty=1.15)
     

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so


/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: /usr/lib64-nvidia did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('/sys/fs/cgroup/memory.events /var/colab/cgroup/jupyter-children/memory.events')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('//172.28.0.1'), PosixPath('http'), PosixPath('8013')}
  warn(msg)
/usr/local/lib/python3.10/dist-packages/bitsandbytes/cuda_setup/main.py:145: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('--logtostderr --listen_host=172.28.0.12 --

CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
CUDA SETUP: CUDA runtime path found: /usr/local/cuda/lib64/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /usr/local/lib/python3.10/dist-packages/bitsandbytes/libbitsandbytes_cuda118.so...


Loading checkpoint shards:   0%|          | 0/14 [00:00<?, ?it/s]

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [ ]:
pip install xformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.2/108.2 MB 11.4 MB/s eta 0:00:00


In [ ]:
class customLLM(LLM):
    def _call(self, prompt, stop=None):
        res = pipeline(prompt)
        prompt_length = len(prompt)
        return res[0]["generated_text"][prompt_length:] 
    def _identifying_params(self):
        return {"name_of_model": "koala-7b"}
    def _llm_type(self):
        return "custom"

In [ ]:
print(customLLM()._call("Tell me somthing about New York City."))




A: Sure, I can tell you something about New York City! It's a city located in the state of New York on the East Coast of the United States. It is known for its iconic landmarks such as the Empire State Building and Times Square, as well as its diverse culture and vibrant energy. The city has a rich history and is home to many world-class museums, galleries, and restaurants. If you ever have the chance to visit New York City, be sure to check out some of these attractions and experience all that this amazing city has to offer!


In [ ]:
embed_model = LangchainEmbedding(HuggingFaceEmbeddings())


In [ ]:
max_input_size = 512
num_output = 200
max_chunk_overlap = 20
chunk_size_limit = 200

llm_predictor = LLMPredictor(llm=customLLM())

In [ ]:
prompt_helper = PromptHelper(max_input_size, num_output,max_chunk_overlap,chunk_size_limit=chunk_size_limit)
service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, embed_model=embed_model, prompt_helper=prompt_helper, chunk_size_limit = chunk_size_limit)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!pip -q install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 16.5 MB/s eta 0:00:00


In [ ]:
documents = SimpleDirectoryReader('./data').load_data()
index = GPTSimpleVectorIndex.from_documents(documents,service_context=service_context)
index.save_to_disk('index.json')

In [ ]:
query_text = "in Backup and Recovery, what Customer Data is ?"
response = index.query(query_text,response_mode="compact",service_context=service_context, similarity_top_k=1)
print(response)


Answer: Customer Data refers to the data that belongs to A vataar's customers and is stored in the secure production account in A WS Cloud using a combination of Amazon S3.


In [ ]:
query_text = "from where we can access files stored on Smallpdf ?"
response = index.query(query_text,response_mode="compact",service_context=service_context, similarity_top_k=1)
print(response)


Answer: From anywhere, including your computer, phone, or tablet.
